In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as wi

In [ ]:
def kernel_mult(pop,jump,coef=None):
    if coef is None:
        coef = 1.0/(2*jump+1)/(2*jump+1)
    if jump==0:
        npop = pop.copy()
        return coef*npop
    # Create teselated pop
    pop2 = np.zeros((pop.shape[0]+2*jump,pop.shape[1]+2*jump))
    pop2[jump:-jump,jump:-jump] = pop
    # Sides
    pop2[:jump,jump:-jump] = pop[-jump:,:]
    pop2[-jump:,jump:-jump] = pop[:jump,:]
    pop2[jump:-jump,:jump] = pop[:,-jump:]
    pop2[jump:-jump,-jump:] = pop[:,:jump]
    # Corners
    pop2[:jump,:jump] = pop[-jump:,-jump:]
    pop2[-jump:,:jump] = pop[:jump,-jump:]
    pop2[:jump,-jump:] = pop[-jump:,:jump]
    pop2[-jump:,-jump:] = pop[:jump,:jump]
    # Update pops:
    npop = np.zeros(pop.shape)
    for x in range(2*jump+1):
        for y in range(2*jump+1):
            npop += coef*pop2[x:x+pop.shape[0],y:y+pop.shape[1]]
    return npop

def update_model(ha,ly,
            ha_breed=0.05,ly_effi=0.4,ly_dead=0.05,hunt_rate=0.01,mode='mult',
            ha_jump=1,ly_jump=1):
    ha = ha.copy()
    ly = ly.copy()
    # ly attack
    if mode=='exp':
        surv_frac = np.exp(-hunt_rate*ly[ha>0]/ha[ha>0])
        hunts = np.zeros(ha.shape)
        hunts[ha>0] = ha[ha>0]*(1.0-surv_frac)
        ha -= hunts
    elif mode=='mult':
        hunts = ly*ha*hunt_rate
        hunts[hunts>ha] = ha[hunts>ha]
        ha -= hunts
    # ly reproduce
    ly += hunts*ly_effi
    # ha reproduce
    ha *= (1.0+ha_breed)
    # ly die
    ly *= (1.0-ly_dead)
    # ha move
    ha = kernel_mult(ha,ha_jump)
    # ly move
    ly = kernel_mult(ly,ly_jump)
    #
    return ha,ly,hunts

In [ ]:
SIZE = 40
INIHA = 400
INILY = 50
STEPS = 2000
JUMP_DELAY = 20
pop = np.zeros((SIZE,SIZE))

ha_pos = [np.random.random((SIZE,SIZE))*INIHA]
ly_pos = [np.random.random((SIZE,SIZE))*INILY]
ha_sum = [np.sum(ha_pos)]
ly_sum = [np.sum(ly_pos)]
hu_pos = [np.zeros(ha_pos[0].shape)]
hu_sum = [0]

for t in range(STEPS+1):
    jump = ((t-1)%JUMP_DELAY)==0
    ha,ly,hu = update_model(ha_pos[-1],ly_pos[-1],ha_jump=jump,ly_jump=jump)
    ha_pos.append(ha)
    ly_pos.append(ly)
    hu_pos.append(hu)
    ha_sum.append(np.sum(ha))
    ly_sum.append(np.sum(ly))
    hu_sum.append(np.sum(hu))

def plot_time(t):
    img = np.zeros((ha_pos[0].shape[0],ha_pos[0].shape[1],3))
    img[:,:,2] = ha_pos[t]/(2*INIHA)
    img[:,:,0] = ly_pos[t]/(2*INILY)
    img[img>1.0] = 1.0
    f, axarr = plt.subplots(1,2,figsize=(12,6))
    axarr[0].set_title('Pops. ubication')
    axarr[0].imshow(img)
    axarr[1].set_title('Total pops')
    axarr[1].plot(hu_sum,color=(0.4,0.4,0.0))
    axarr[1].plot(ha_sum,color=(0.0,0.0,0.4))
    axarr[1].plot(ly_sum,color=(0.4,0.0,0.0))
    axarr[1].plot(hu_sum[:t+1],color=(0.1,0.1,0.0))
    axarr[1].plot(ha_sum[:t+1],color=(0.0,0.0,1.0))
    axarr[1].plot(ly_sum[:t+1],color=(1.0,0.0,0.0))
    plt.show()
    print("ha %f"%ha_sum[t])
    print("ly %f"%ly_sum[t])
    print("hu %f"%hu_sum[t])

t_slider = wi.IntSlider(min=0,max=len(ha_pos)-2,continuous_update=False)
interactive_plot = wi.interact(plot_time,t=t_slider)
interactive_plot.widget.layout.height = '500px'
interactive_plot;